In [342]:
#initiate skills dictionay that contains each skill and variations of its name
#have an issue with R since the letter will mostly in the job description
#does capitalization matter?
skills = {
    'Excel' : ['Excel','MSExcel'],
    'SQL' : ['mysql', 'MS SQL', 'SQL', 'noSQL', 'NoSQL', 'NOSQL'],
    'Python': ['python','Python'],
    'JavaScript':['JS','JavaScript','javascript'],
    'pandas': ['Pandas','pandas'],
    'Hadoop': ['Hadoop'],
    'MongoDB': ['Mongo','MongoDB'],
    'Spark': ['Spark'],
    'Leaflet':['Leafelet','leaflet'],
    'Numpy':['numpy','Numpy'],
    'VBA':['vba','VBA'],
    'd3':['d3','D3'],
    'Tableau':['Tableau','tableau'],
    'R': ['RStudio'],
    'Machine Learning':['Machine Learning','Machine learning','machine learning', 'Tensorflow']
}

In [218]:
#import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
import time
import numpy as np
from collections import Counter
from config import gkey
import requests
import json

In [219]:
#funtion for browser
def init_browser(): 
    executable_path = {'executable_path':'./chromedriver'}
    return Browser('chrome',**executable_path, headless=False)

In [354]:
#Create an instance of a broswer

indeed_url = "https://www.indeed.com/jobs?as_and=data+science&as_phr=&as_any=&as_not=&as_ttl=data+scientist&as_cmp=&jt=all&st=&as_src=&salary=&radius=0&l=CA&fromage=any&limit=50&sort=&psf=advsrch"
#url for data analyst (entry)
#url = "https://www.indeed.com/jobs?as_and=data+analyst&as_phr=&as_any=&as_not=&as_ttl=data+analyst&as_cmp=&jt=all&st=&as_src=&salary=&radius=5&l=CA&fromage=any&limit=50&sort=date&psf=advsrch"

#url = "https://www.indeed.com/jobs?q=(Python+or+SQL+or+MySQL+or+Tableau+or+VBA+or+Excel+or+Pandas+or+Javascript+or+D3+or+Splinter+or+JSON+or+Machine+or+Learning+or+Leaflet+or+GeoJson)+-machinist,+-CNC,&l=CA&radius=0&jt=fulltime&explvl=mid_level&sort=date&limit=50&st=employer&fromage=3"
#url = "https://www.indeed.com/jobs?q=data+analyst+(Python+or+SQL+or+MySQL+or+Tableau+or+VBA+or+Excel+or+Pandas+or+Javascript+or+D3+or+web-scraping+or+JSON+or+Machine+or+Learning+or+Leaflet+or+GeoJson)+-machinist,+-CNC,&l=CA&radius=0&jt=fulltime&explvl=mid_level&sort=date&limit=50&st=employer&fromage=3"
browser = init_browser()
browser.driver.set_window_size(1680, 1050)
browser.visit(indeed_url)
html= browser.html

In [355]:
#creat soup object
soup = bs(html, "html.parser")

In [356]:
#Find all divs which contain a job posting
#find the link of each job posting save them to a list. instead of registering the full link, only store
#the first 50 characters (that should be enough to create uniqueness between postings)
div_results = soup.find_all('div', {'class':'row result clickcard'})
a_tags = []

div_results
for div in div_results:
    a = div.a.get('href')
    a_tags.append(a[:50])
    #print(a[:50])
    #print("------------------------------------------------------------------------------------------")

In [357]:
#print length of tags list
page_tag_count = len(a_tags) 
print(page_tag_count)
#print(a_tags)
a_tags

56


['/pagead/clk?mo=r&ad=-6NYlbfkN0CFSg06BLvPbR-HAeeUdX',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0BZGVbs71v5EJ1Sm8QuTI',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntbkP',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0CFSg06BLvPbR-HAeeUdX',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0AR_qGNZB4jkM1DWpdIph',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntbkP',
 '/pagead/clk?mo=r&ad=-6NYlbfkN0CleyozbRSfLCtZHtSeSH',
 '/company/Bridgepoint-Education/jobs/Junior-Data-Sc',
 '/rc/clk?jk=5c99379f444efe0c&fccid=c1099851e9794854',
 '/rc/clk?jk=56e9ad66b5f392da&fccid=ddae355770caf5ef',
 '/company/M-Theory-Solutions/jobs/Data-Scientist-44',
 '/rc/clk?jk=273ff3a4c6fde5b6&fccid=8b219f1987ef65ea',
 '/rc/clk?jk=7f1f5a6292581fc7&fccid=68ec8a8ee6cf5ac6',
 '/rc/clk?jk=4084e8caad382bd3&fccid=128451ec69a4c285',
 '/company/Lumen-Solutions-Inc/jobs/Data-Scientist-C',
 '/rc/clk?jk=31a1a5e5ac57629c&fccid=16d24b4316c5d97f',
 '/rc/clk?jk=1bfbefe559bc942f&fccid=2431bcc75b874651',
 '/rc/clk?jk=944268a7d74bafba&fccid=fbd294446d1ea4d4',
 '/rc/clk?

In [358]:

counter = 0

# Initialize empty dictionarys and lists to dump into JSON objects
x = {}
indeed_job_data = []
locations = []
skills_count = {}
target_city = []
skills_list = []


# Loop through a_tag content and click on job postings
for tag in a_tags:
    browser.find_link_by_partial_href(tag).click()
    time.sleep(1)
    html2 = browser.html
    soup2 = bs(html2, "html.parser")
    
    try:
        job_desc = soup2.find('div', {'id':'vjs-tab-job'}).get_text()
        company = soup2.find('span', {'id':'vjs-cn'}).get_text()
        title = soup2.find('div', {'id':'vjs-jobtitle'}).get_text()
        location = soup2.find('span', {'id':'vjs-loc'}).get_text().split(" - ")[1].split(",")[0]
        for skill, terms in skills.items():
            for term in terms:
                if term in job_desc:
                        skills_list.append(term)
                break
        
        target_city = location
        target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
    "address=%s&key=%s" % (target_city, gkey)
        req = requests.get(target_url)
        res = req.json()
        result = res['results'][0]

        
        geodata = []
        lat = result['geometry']['location']['lat']
        long = result['geometry']['location']['lng']
        
        x = { 
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lat, long]
            },
            "properties": {
            "website" : 'Indeed',
            "job title" : title,
            "job_posting_url" : indeed_url,
            "job_scrape_date": "10/25/18",
            "job_posting_date": "2 days ago",
            'company': company,
            'skills' : skills_list,
            'city' : location
                }
            }
        
        indeed_job_data.append(x) 

    except AttributeError as ex:
        print(ex)
        print('error at ', counter)
        
    counter += 1
    location = []
    skills_list = []
    x = {}
    data = {}

    print(counter)
 



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
'NoneType' object has no attribute 'get_text'
error at  41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [379]:
indeed_job_data[5]['properties']['skills']


['Excel', 'Hadoop', 'Machine Learning']

In [380]:
indeed_skills_count = {}
for skill, terms in skills.items():
    indeed_skills_count[skill]=0

count = 0

for entry in indeed_job_data:
    for skill, terms in skills.items():
        for term in terms:
            if term in entry['properties']['skills']:

                indeed_skills_count[skill] +=1
                count +=1

                break
indeed_skills_count


{'Excel': 17,
 'SQL': 0,
 'Python': 2,
 'JavaScript': 2,
 'pandas': 3,
 'Hadoop': 14,
 'MongoDB': 1,
 'Spark': 8,
 'Leaflet': 0,
 'Numpy': 1,
 'VBA': 0,
 'd3': 1,
 'Tableau': 13,
 'R': 0,
 'Machine Learning': 17}

In [385]:
#url = "https://www.monster.com/jobs/search/?q=data-scientist&where=CA&intcid=skr_navigation_nhpso_searchMain&jobid=201469712"
monster_url = "https://www.monster.com/jobs/search/Full-Time_8?q=data-scientist&where=CA&tm=3&jobid=202164956"

In [386]:
browser = init_browser()
browser.driver.set_window_size(1680, 1050)
browser.visit(monster_url)
html= browser.html


In [387]:
soup = bs(html, "html.parser")

In [388]:
#Find all divs which contain a job posting
#find the link of each job posting save them to a list. instead of registering the full link, only store
#the first 50 characters (that should be enough to create uniqueness between postings)
soup = bs(html, "html.parser")
div_results = soup.find_all('div', {'class':'summary'})

a_tags = []

div_results
for div in div_results:
    a = div.a.get('href')
    a_tags.append(a[:70])
    print(a[:70])
    print("------------------------------------------------------------------------------------------")

https://job-openings.monster.com/data-scientist-san-jose-ca-us-collabe
------------------------------------------------------------------------------------------
https://job-openings.monster.com/senior-data-scientist-san-francisco-c
------------------------------------------------------------------------------------------
https://job-openings.monster.com/senior-data-scientist-san-francisco-c
------------------------------------------------------------------------------------------
https://job-openings.monster.com/data-scientist-san-diego-ca-us-ablefo
------------------------------------------------------------------------------------------
https://job-openings.monster.com/data-scientist-san-diego-ca-us-apex-s
------------------------------------------------------------------------------------------
https://job-openings.monster.com/data-architect-data-science-san-jose-
------------------------------------------------------------------------------------------
https://job-openings.monster

In [389]:
#print length of tags list
page_tag_count = len(a_tags) 
print(page_tag_count)
#print(a_tags)
a_tags

25


['https://job-openings.monster.com/data-scientist-san-jose-ca-us-collabe',
 'https://job-openings.monster.com/senior-data-scientist-san-francisco-c',
 'https://job-openings.monster.com/senior-data-scientist-san-francisco-c',
 'https://job-openings.monster.com/data-scientist-san-diego-ca-us-ablefo',
 'https://job-openings.monster.com/data-scientist-san-diego-ca-us-apex-s',
 'https://job-openings.monster.com/data-architect-data-science-san-jose-',
 'https://job-openings.monster.com/sr-manager-data-science-san-diego-ca-',
 'https://job-openings.monster.com/associate-sales-consultant-stockton-c',
 'https://job-openings.monster.com/senior-devops-engineer-santa-barbara-',
 'https://job-openings.monster.com/senior-front-end-engineer-san-francis',
 'https://job-openings.monster.com/watson-content-analytics-developer-fr',
 'https://job-openings.monster.com/research-associate-san-diego-ca-us-ra',
 'https://job-openings.monster.com/watson-content-analytics-developer-fr',
 'https://job-openings.mo

In [390]:

counter = 0

# Initialize empty dictionarys and lists to dump into JSON objects
x = {}
monster_job_data = []
locations = []
skills_count = {}
target_city = []
skills_list = []


# Loop through a_tag content and click on job postings
for tag in a_tags:
    browser.find_link_by_partial_href(tag).click()
    time.sleep(1)
    html2 = browser.html
    soup2 = bs(html2, "html.parser")

    
    try:
        # set variables to dump into JSON 
        location = soup2.find('h2', {'class':'subtitle'}).get_text().split(",")[0].split(",")[0]
        job_desc = soup2.find('div', {'class':'details-content'}).get_text()
        title = soup2.find('h1', {'class':'title'}).get_text().strip()


        
        for skill, terms in skills.items():
            for term in terms:
                if term in job_desc:
                        skills_list.append(term)
                break
        
        target_city = location
        target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
    "address=%s&key=%s" % (target_city, gkey)
        req = requests.get(target_url)
        res = req.json()
        result = res['results'][0]

        lat = result['geometry']['location']['lat']
        long = result['geometry']['location']['lng']
        
        x = { 
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lat, long]
            },
            "properties": {
            "website" : 'Monster',
            "job title" : title,
            "job_posting_url" : monster_url,
            "job_scrape_date": "10/25/18",
            "job_posting_date": "2 days ago",
            'company': "NA",
            'skills' : skills_list,
            'city' : location
                }
            }
        
        monster_job_data.append(x) 

    except AttributeError as ex:
        print(ex)
        print('error at ', counter)
        
    counter += 1
    location = []
    skills_list = []
    x = {}
    data = {}

    print(counter)


 

1
2
3
4
5
6
7
8
'NoneType' object has no attribute 'get_text'
error at  8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [391]:
monster_job_data[1]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [37.7749295, -122.4194155]},
 'properties': {'website': 'Monster',
  'job title': 'Senior Data Scientist at Clandica Inc',
  'job_posting_url': 'https://www.monster.com/jobs/search/Full-Time_8?q=data-scientist&where=CA&tm=3&jobid=202164956',
  'job_scrape_date': '10/25/18',
  'job_posting_date': '2 days ago',
  'company': 'NA',
  'skills': ['python'],
  'city': 'San Francisco'}}

In [392]:
monster_skills_count = {}
for skill, terms in skills.items():
    monster_skills_count[skill]=0


count = 0

for entry in monster_job_data:
    for skill, terms in skills.items():
        for term in terms:
            if term in entry['properties']['skills']:

                monster_skills_count[skill] +=1
                count +=1

                break
monster_skills_count

{'Excel': 6,
 'SQL': 0,
 'Python': 2,
 'JavaScript': 3,
 'pandas': 1,
 'Hadoop': 7,
 'MongoDB': 0,
 'Spark': 2,
 'Leaflet': 0,
 'Numpy': 0,
 'VBA': 0,
 'd3': 0,
 'Tableau': 5,
 'R': 0,
 'Machine Learning': 0}

In [32]:
# this code will advance the scraping to the next jobs page and close pop-up
browser.click_link_by_partial_text('Next')
browser.click_link_by_id('popover-close-link')


{}

In [393]:
linkedin_url = "https://www.linkedin.com/jobs/search/?distance=25&f_E=2%2C3&f_GC=us.7-1-0-38-1%2Cus.7-1-0-37-41%2Cus.7-1-0-19-57%2Cus.7-1-0-43-16%2Cus.7-1-0-34-21%2Cus.7-1-0-30-18&f_T=25190&f_TP=1%2C2&keywords=data%20scientist"

In [398]:
browser = init_browser()
browser.driver.set_window_size(1680, 1050)
browser.visit(linkedin_url)
html= browser.html

In [399]:
soup = bs(html, "html.parser")
div_results = soup.find_all('li', {'itemprop':'itemListElement'})

a_tags = []

div_results
for div in div_results:
    a = div.a.get('href')
    a_tags.append(a[:70])
    print(a[:70])
    print("------------------------------------------------------------------------------------------")

https://www.linkedin.com/jobs/view/data-scientist-management-consultan
------------------------------------------------------------------------------------------
https://www.linkedin.com/jobs/view/data-scientist-product-analytics-at
------------------------------------------------------------------------------------------
https://www.linkedin.com/jobs/view/data-scientist-at-zumper-930975483?
------------------------------------------------------------------------------------------
https://www.linkedin.com/jobs/view/data-scientist-at-envoy-930973938?t
------------------------------------------------------------------------------------------
https://www.linkedin.com/jobs/view/data-scientist-growth-at-smartnews-
------------------------------------------------------------------------------------------
https://www.linkedin.com/jobs/view/data-scientist-at-square-up-media-9
------------------------------------------------------------------------------------------
https://www.linkedin.com/job

In [400]:
#print length of tags list
page_tag_count = len(a_tags) 
print(page_tag_count)

a_tags

25


['https://www.linkedin.com/jobs/view/data-scientist-management-consultan',
 'https://www.linkedin.com/jobs/view/data-scientist-product-analytics-at',
 'https://www.linkedin.com/jobs/view/data-scientist-at-zumper-930975483?',
 'https://www.linkedin.com/jobs/view/data-scientist-at-envoy-930973938?t',
 'https://www.linkedin.com/jobs/view/data-scientist-growth-at-smartnews-',
 'https://www.linkedin.com/jobs/view/data-scientist-at-square-up-media-9',
 'https://www.linkedin.com/jobs/view/data-scientist-at-directly-92432803',
 'https://www.linkedin.com/jobs/view/data-scientist-at-grand-rounds-inc-',
 'https://www.linkedin.com/jobs/view/data-scientist-at-spin-electric-sco',
 'https://www.linkedin.com/jobs/view/data-scientist-at-nutanix-930528785',
 'https://www.linkedin.com/jobs/view/data-scientist-at-goat-sneaker-mark',
 'https://www.linkedin.com/jobs/view/data-scientist-at-vevo-917574663?tr',
 'https://www.linkedin.com/jobs/view/senior-data-scientist-at-pocket-gem',
 'https://www.linkedin.co

In [401]:

counter = 0

# Initialize empty dictionarys and lists to dump into JSON objects
x = {}
linkedin_job_data = []
locations = []
skills_count = {}
target_city = []
skills_list = []

# Loop through a_tag content and click on job postings
for tag in a_tags:

    browser.find_link_by_partial_href(tag).click()
    time.sleep(1)
    html2 = browser.html
    soup2 = bs(html2, "html.parser")

    try:
        # set variables to dump into JSON 
        location = soup2.find('h3', {'class':'location'}).get_text().split(",")[0]
        company = soup2.find('span', {'class':'company'}).get_text()
        job_desc = soup2.find('div', {'class':'description-section'}).get_text()
        title = soup2.find('h1', {'class':'title'}).get_text().strip()
        
        for skill, terms in skills.items():
            for term in terms:
                if term in job_desc:
                        skills_list.append(term)
                break
        
        target_city = location
        target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
    "address=%s&key=%s" % (target_city, gkey)
        req = requests.get(target_url)
        res = req.json()
        result = res['results'][0]

        lat = result['geometry']['location']['lat']
        long = result['geometry']['location']['lng']
        
        x = { 
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lat, long]
            },
            "properties": {
            "website" : 'linkedin',
            "job title" : title,
            "job_posting_url" : linkedin_url,
            "job_scrape_date": "10/25/18",
            "job_posting_date": "2 days ago",
            'company': "NA",
            'skills' : skills_list,
            'city' : location
                }
            }
        
        linkedin_job_data.append(x) 

    except WebDriverException as ex:
        print(ex)
        print('error at ', counter)
        
    browser.back()
    
    counter += 1
    location = []
    skills_list = []
    x = {}
    data = {}

    print(counter)
   # print(job_data)  

    #job_json.append(job_data)  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [402]:
linkedin_job_data[1]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [37.7749295, -122.4194155]},
 'properties': {'website': 'linkedin',
  'job title': 'Data Scientist, Product Analytics',
  'job_posting_url': 'https://www.linkedin.com/jobs/search/?distance=25&f_E=2%2C3&f_GC=us.7-1-0-38-1%2Cus.7-1-0-37-41%2Cus.7-1-0-19-57%2Cus.7-1-0-43-16%2Cus.7-1-0-34-21%2Cus.7-1-0-30-18&f_T=25190&f_TP=1%2C2&keywords=data%20scientist',
  'job_scrape_date': '10/25/18',
  'job_posting_date': '2 days ago',
  'company': 'NA',
  'skills': ['Excel', 'Tableau'],
  'city': 'San Francisco'}}

In [404]:
linkedin_skills_count = {}
for skill, terms in skills.items():
    linkedin_skills_count[skill]=0

count = 0

for entry in linkedin_job_data:
    for skill, terms in skills.items():
        for term in terms:
            if term in entry['properties']['skills']:

                linkedin_skills_count[skill] +=1
                count +=1

                break
linkedin_skills_count

{'Excel': 9,
 'SQL': 0,
 'Python': 0,
 'JavaScript': 1,
 'pandas': 1,
 'Hadoop': 4,
 'MongoDB': 0,
 'Spark': 8,
 'Leaflet': 0,
 'Numpy': 2,
 'VBA': 0,
 'd3': 1,
 'Tableau': 4,
 'R': 0,
 'Machine Learning': 5}

In [405]:
linkedin_job_data[1]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [37.7749295, -122.4194155]},
 'properties': {'website': 'linkedin',
  'job title': 'Data Scientist, Product Analytics',
  'job_posting_url': 'https://www.linkedin.com/jobs/search/?distance=25&f_E=2%2C3&f_GC=us.7-1-0-38-1%2Cus.7-1-0-37-41%2Cus.7-1-0-19-57%2Cus.7-1-0-43-16%2Cus.7-1-0-34-21%2Cus.7-1-0-30-18&f_T=25190&f_TP=1%2C2&keywords=data%20scientist',
  'job_scrape_date': '10/25/18',
  'job_posting_date': '2 days ago',
  'company': 'NA',
  'skills': ['Excel', 'Tableau'],
  'city': 'San Francisco'}}